## Chapter 18. asyncio를 이용한 동시성
Concurrency is Not Paralleism!. 일반적인 운용 환경에서도 100개 이상의 프로세스를 일상적으로 실행한다. 진짜 병렬로 처리하려면 CPU 코어가 여러 개 있어야 한다. 따라서 실제로는 대부분의 처리가 동시에 수행되지만 병렬로 수행되지는 않는다. 이 장에서는 이벤트 루프에 의해 운용되는 코루틴을 이용해서 동시성을 구현하는 asyncio 패키지에 대해 설명한다.

### 18.1 스레드와 코루틴 비교
파이선에는 스레드를 종료시키는 API가 정의되지 않지 않기 때문에 여기서는 singal.go 속성을 사용했다. 

In [1]:
""" [예제 18-1] spinner_thread.py : 스레드로 텍스트 스피너 애니메이트하기 """
import threading
import itertools
import time
import sys

class Signal:
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) # 문자열 길이만큼 백스페이스를 반복해서 커서를 앞으로 이동시킴
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status)) # 스페이스로 기존 문자열을 지우고 백스페이스로 원래 위치로 돌아감
    
def slow_function():
    # 입출력을 위해 장시간 기다리는 것 처럼 보임
    time.sleep(3) # 주 스레드에서 sleep( ) 함수를 호출할 때 GIL이 해제되므로 두 번째 스레드가 진행됨
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start() # spinner 스레드 시작
    result = slow_function()
    signal.go = False
    spinner.join()  # spinner 스레드가 끝날 때 까지 기다림
    return result

def main():
    result = supervisor()
    print('Answer:', result)
    
if __name__ == '__main__':
    main()

spinner object: <Thread(Thread-4, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          Answer: 42


스레드 대신 @asyncio.coroutine을 이용해서 동일한 동작을 어떻게 구현할 수 있는지 알아보자
```
asyncio는 '코루틴'을 더욱 엄격히 정의한다. asyncio API에 사용할 코루틴은 본체 안에서 yield가 아니라 yield from을 사용해야 한다. 그리고 asyncio 코루틴은 yield from으로 호출하는 호출자에게 구동되거나, 이 장에서 설명할 asyncio.async() 등의 함수에 전달해서 구동해야 한다. 마지막으로 예제에서 보여주는 것러첨 @asyncio.coroutine 데커레이터를 코루틴에 적용해야 한다. 
```


In [2]:
""" [예제 18-2] spinner_asyncio.py : 코루틴으로 텍스트 스피너 애니메이트 하기 """

import asyncio
import itertools
import sys

@asyncio.coroutine # 사용할 코루틴을 데커레이트 처리함
def spin(msg):     # 예제 18-1의 singal 인수가 필요없음
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) 
        try:
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status)) 

@asyncio.coroutine
def slow_function():
    yield from asyncio.sleep(3)
    return 42
    
@asyncio.coroutine
def supervision():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervision())
    loop.close()
    print('Answer', result)
    
if __name__ == '__main__':
    main()

RuntimeError: Event loop is running.

spinner object: <Task pending coro=<spin() running at <ipython-input-2-ef8aa0b38a1a>:7>>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
